<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
!pip install simpy

In [5]:
import simpy
import pandas as pd

# Service times for each station in seconds
service_times = [25, 15, 34, 37, 38, 56, 25, 15, 19, 25, 17, 17, 7]

# Number of service stations
num_stations = len(service_times)

# Number of customers to serve
num_customers = 500

# Simulation environment
class ServiceStation:
    def __init__(self, env, service_time):
        self.env = env
        self.server = simpy.Resource(env, capacity=1)
        self.service_time = service_time

    def serve_customer(self):
        yield self.env.timeout(self.service_time)

def customer(env, stations):
    for station in stations:
        with station.server.request() as request:
            yield request
            yield env.process(station.serve_customer())

# Create simulation environment
env = simpy.Environment()

# Create service stations
stations = [ServiceStation(env, service_time) for service_time in service_times]

# Create customer process
env.process(customer(env, stations))

# Run the simulation
env.run()

# Calculate the average service time for each station
average_service_times = [station.service_time for station in stations]

# Calculate the required number of servers at each station
required_servers = [num_customers / time for time in average_service_times]

# Create a DataFrame for the results
results = pd.DataFrame({
    "Service Station": range(1, num_stations+1),
    "Average Service Time (seconds)": average_service_times,
    "Required Servers": required_servers
})

print(results)


    Service Station  Average Service Time (seconds)  Required Servers
0                 1                              25         20.000000
1                 2                              15         33.333333
2                 3                              34         14.705882
3                 4                              37         13.513514
4                 5                              38         13.157895
5                 6                              56          8.928571
6                 7                              25         20.000000
7                 8                              15         33.333333
8                 9                              19         26.315789
9                10                              25         20.000000
10               11                              17         29.411765
11               12                              17         29.411765
12               13                               7         71.428571


In [16]:
import simpy
import pandas as pd

# Service times for each station in seconds
service_times = [25, 15, 34, 37, 38, 56, 25, 15, 19, 25, 17, 17, 7]

# Desired simulation time in seconds (7 hours)
simulation_time = 7 * 60 * 60  # 7 hours * 60 minutes * 60 seconds

# Number of customers to serve
num_customers = 1000

# Simulation environment
class ServiceStation:
    def __init__(self, env, service_time):
        self.env = env
        self.service_time = service_time
        self.server = None  # Will be created later

    def serve_customer(self):
        yield self.env.timeout(self.service_time) #Generates the simulation time. 

# Create simulation environment
env = simpy.Environment()

# Create service stations
stations = [ServiceStation(env, service_time) for service_time in service_times]

# Distribute total available servers (22) among stations based on service times
total_servers = 22
total_service_times = sum(service_times)
for i, station in enumerate(stations):
    required_servers = max(1, int((service_times[i] / total_service_times) * total_servers))
    station.server = simpy.Resource(env, capacity=required_servers)

# Create customer process
def customer(env, stations):
    served_customers = 0
    while served_customers < num_customers:
        for station in stations:
            with station.server.request() as request:
                yield request
                yield env.process(station.serve_customer())
                served_customers += 1

env.process(customer(env, stations))

# Run the simulation
env.run(until=simulation_time)

# Calculate the average service time for each station
average_service_times = [station.service_time for station in stations]

# Calculate the required number of servers at each station
required_servers = [station.server.capacity for station in stations]

# Create a DataFrame for the results
results = pd.DataFrame({
    "Service Station": range(1, len(stations) + 1),
    "Average Service Time (seconds)": average_service_times,
    "Required Servers": required_servers
})

print(results)



    Service Station  Average Service Time (seconds)  Required Servers
0                 1                              25                 1
1                 2                              15                 1
2                 3                              34                 2
3                 4                              37                 2
4                 5                              38                 2
5                 6                              56                 3
6                 7                              25                 1
7                 8                              15                 1
8                 9                              19                 1
9                10                              25                 1
10               11                              17                 1
11               12                              17                 1
12               13                               7                 1


# Code Explanation:
    



__simpy:__ A discrete event simulation framework used to model and simulate the system.
pandas as pd: The pandas library is used to manipulate and analyze data efficiently.
Define service times for each station:

__service_times:__ A list containing the service times (in seconds) required by each service station.
Set up simulation parameters:

simulation_time: The total simulation time in seconds. Here, it's set to 7 hours.
num_customers: The number of customers to serve during the simulation.
Define the ServiceStation class:
The class represents a service station and its behavior.


__ __init__ __ : Initializes the service station with an environment (env) and a specific service_time.
serve_customer: A generator function that simulates the service time by yielding a timeout event.
Create the simulation environment:

__env = simpy.Environment():__ Creates a simulation environment using SimPy.
Create service stations:

__stations:__  A list of ServiceStation instances, each with its corresponding service time.
Distribute available servers among stations:

__total_servers:__ The total number of available servers.
Calculate the sum of all service times to distribute servers proportionally.
For each station, calculate the required number of servers based on its proportion of service time compared to the total service times. At least one server is allocated per station.
Create simpy.Resource instances (representing servers) for each station with the calculated capacity.
Define the customer process:

__customer:__ A generator function that represents a customer arriving at different stations.
The process runs continuously until the desired number of customers is served.
The customer iterates through each station, requesting a server, serving the customer, and updating the served count.
Set up the customer process in the simulation environment:

__env.process(customer(env, stations)):__ Initializes the customer process in the simulation environment.
Run the simulation:

__env.run(until=simulation_time):__ Executes the simulation until the specified simulation_time.
Calculate average service times and required servers:

__average_service_times:__ A list containing the original service times of each station.
required_servers: A list containing the required number of servers at each station.
Create a DataFrame for the simulation results:

__results:__ A pandas DataFrame containing the following columns:
"Service Station": The index of each station.
"Average Service Time (seconds)": The original service times for each station.
"Required Servers": The required number of servers at each station.
Print the results:

print(results): Displays the simulation results in tabular form.